In [85]:
import cv2 as cv
import numpy as np

# Mediapipe Hands Documentation fund at https://google.github.io/mediapipe/solutions/hands.html
import mediapipe as mp
mp_hands = mp.solutions.hands

In [89]:
def getHands(current, width, height):
  foundHands = []
  center = (0,0)  
  if current.multi_hand_landmarks:
    # iterates througheach hand
    for hand_no, hand in enumerate(current.multi_hand_landmarks):
      handedness = current.multi_handedness[hand_no].classification.pop(0).label
      detectedHands = []
      for landmark in hand.landmark:
        detectedHands.append((int(landmark.x * width), int(landmark.y * height), int(landmark.z * width)))
      #Calculate the min/max x and y values to draw a bounding box around the hand
      xMin = int(np.min(np.array(detectedHands)[:, 0]))
      yMin = int(np.min(np.array(detectedHands)[:, 1]))
      xMax = int(np.max(np.array(detectedHands)[:, 0]))
      yMax = int(np.max(np.array(detectedHands)[:, 1]))
      #Calculate the center of the boundign box, to track the hand
      center = (xMin + (xMax-xMin)//2, yMin + (yMax-yMin)//2)
      foundHands.append(((xMin, yMin), (xMax, yMax),center, handedness))
  
  return foundHands
def capture():
    # video = cv.VideoCapture(0)
    video = cv.VideoCapture("./videos_partA.mp4")
    running, original = video.read()
    height, width, _ = original.shape

    out = cv.VideoWriter('out.avi',cv.VideoWriter_fourcc('M','J','P','G'), video.get(cv.CAP_PROP_FPS), (width,height))

    # Initilize Mediapipe hands
    hands = mp.solutions.hands.Hands(min_detection_confidence=0.5, min_tracking_confidence=0.25)

    positions = {'Left': [], 'Right' :[]}

    while True:
      running, frame = video.read()
      if not running:
        break
      # hand processing
      tracker = hands.process(frame)
      detectedHands = getHands(tracker, width, height)

      for hand in detectedHands:
        cv.rectangle(frame, hand[0], hand[1], (0,255,0), 2)
        if hand[3] in positions:
          positions[hand[3]].append(hand[2])



      for pos in positions:
        if len(positions[pos]) >1:
          pointsInside = positions[pos]
          for index, item in enumerate(pointsInside): 
            if index == len(pointsInside) -1:
              break
            if pos == 'Left':
              cv.line(frame, item, pointsInside[index + 1], [255, 0,0], 2) 
            elif pos == 'Right':
              cv.line(frame, item, pointsInside[index + 1], [255, 0,190], 2)
      
      cv.imshow("Video Feed", frame)
      out.write(frame)
      k = cv.waitKey(1) & 0xff
      if k == 27:
        break

    video.release()
    out.release()
    cv.destroyAllWindows()
capture()
 



: 